In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df_global = pd.read_csv('2020.csv',sep=';')
df_places = pd.read_csv('lieux-2020.csv',sep=';')
df_vehicles = pd.read_csv('vehicules-2020.csv',sep=';')
df_carac = pd.read_csv('caracteristiques-2020.csv',sep=';')
df_users = pd.read_csv('usagers-2020.csv',sep=';')

In [ ]:
df_global = pd.read_csv('2020.csv',sep=';')

In [ ]:
print(df_global[df_global['Id_accident']=='66\xa0505\xa0400'].Id_accident)

In [ ]:
df_global['Id_accident'] = df_global['Id_accident'].str.replace('\xa0','')

In [ ]:
df_global['Id_accident'] = df_global['Id_accident'].astype('int64')

In [ ]:
df_global.head(2)

In [ ]:
df_places.Num_Acc.describe()

In [ ]:
df_vehicles.columns

In [ ]:
df_carac.columns

In [ ]:
df_users.columns

In [ ]:
df_global.select_dtypes('object').columns

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})

In [ ]:
sns.countplot(x='Lettre Conventionnelle Véhicule', data=df_global)

In [ ]:
sns.countplot(x='Lieu Admin Actuel - Territoire Nom', data=df_global)

In [ ]:
sns.countplot(x='Type Accident - Libellé', data=df_global)

In [ ]:
sns.countplot(x='Catégorie véhicule', data=df_global)

## Localisation

On s'intéresse au coordonées GPS des accidents, qui sont dans les fichier **caractéristiques**

In [ ]:
df_carac = pd.read_csv('caracteristiques-2020.csv',sep=';',encoding='latin')

In [ ]:
df_carac.columns

In [ ]:
seps = {x:',' for x in range(2005,2009)}\
        | {2009:'\t'}\
        | {x:',' for x in range(2010,2019)}\
#        | {x:';' for x in range(2019,2021)}
dtype = {'lat':str,'long':str,'an':int}
df_carac = pd.concat([pd.read_csv('data/caracteristiques_' + str(year) + '.csv',sep=sep, dtype=dtype,
                             encoding='latin',on_bad_lines='skip') for (year,sep) in seps.items()])

In [ ]:
df_carac.head()

In [ ]:
df_carac['an'].describe()

On doit corriger les années numérotées de 5 à 17 pour les ramener après 2000

In [ ]:
df_carac.loc[df_carac['an'] < 2000,'an'] += 2000

In [ ]:
df_carac['an']

In [ ]:
df_carac.columns

In [ ]:
df_carac.dtypes

In [ ]:
df_carac = df_carac[(df_carac['long']=='-') == False]
df_carac = df_carac[(df_carac['long']=='0') == False]

In [ ]:
df_carac[['long','lat']].describe()

In [ ]:
def convert_deg(x):
    if not isinstance(x, str) or x == '-':
        return np.nan
    if (x[0] == '-'):
        return float(x[:2]+'.'+ x[2:])
    else:
        return float(x[:2]+'.'+ x[2:])

def safe_convert(x):
    try:
        r = convert_deg(x)
        return r
    except:
        return np.nan

In [ ]:
tmp = df_carac.copy()
tmp.loc[:,'long'] = tmp['long'].apply(safe_convert)
tmp.loc[:,'lat'] = tmp['lat'].apply(safe_convert)
tmp['long'].dropna().count()
tmp.head()

In [ ]:
df_carac.loc[:,'long'] = df_carac['long'].apply(convert_deg)
df_carac.loc[:,'lat'] = df_carac['lat'].apply(convert_deg)


In [ ]:
import plotly.express as px

#print(df_carac[df_carac['lat'] > 30]['long'].dropna().count())
#tmp = df_carac.iloc[0:500000].copy()
tmp = df_carac.copy()
tmp = tmp[tmp['lat'] > 30]
tmp = tmp.dropna().copy()
print(tmp.shape)
tmp = tmp[tmp['an']==2016].copy()

In [ ]:
fig = px.scatter_mapbox(tmp, lat="lat", lon="long",
                        hover_name="Num_Acc", color_discrete_sequence=["blue"], zoom=5, height=850, width=850,
                        center={'lat':46,'lon':2},mapbox_style='open-street-map')

fig.update_layout(mapbox_style="open-street-map")
fig.show()

In [ ]:
plt.figure(figsize=(120,120))
plt.scatter(df_carac['lat'], df_carac['long'])
plt.title('Location of the accident (Metropole)')
plt.show()